In [6]:
import pandas as pd
import numpy as np
import requests as rq
from bs4 import BeautifulSoup as bs
import re
import os
from multiprocessing import Pool
from tqdm import tqdm
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.options import Options


In [11]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import time

options = Options()
options.add_argument('--disable-blink-features=AutomationControlled')
options.add_experimental_option("excludeSwitches", ["enable-automation"])
options.add_experimental_option('useAutomationExtension', False)
options.add_argument("--disable-extensions")

driver = webdriver.Chrome(options=options)
driver.execute_script("Object.defineProperty(navigator, 'webdriver', {get: () => undefined})")

driver.get('https://www.sofascore.com/football/rankings/uefa')
time.sleep(1)  # Esperar a que cargue
ligas=[i['href'] for i in bs(driver.page_source, 'html.parser').find_all('a') if i['href'].startswith('/tournament/football') and 'world' not in i['href'] and 'europe' not in i['href']]
driver.quit()

In [12]:
ligas=list(set(ligas))  # Eliminar duplicados
ligas=['https://www.sofascore.com' + i for i in ligas]  # Añadir el dominio


In [13]:
ligas 

['https://www.sofascore.com/tournament/football/portugal/liga-portugal-betclic/238',
 'https://www.sofascore.com/tournament/football/hungary/nb-i/187',
 'https://www.sofascore.com/tournament/football/italy/serie-a/23',
 'https://www.sofascore.com/tournament/football/cyprus/1-division/171',
 'https://www.sofascore.com/tournament/football/france/ligue-1/34',
 'https://www.sofascore.com/tournament/football/turkey/trendyol-super-lig/52',
 'https://www.sofascore.com/tournament/football/czech-republic/1-liga/172',
 'https://www.sofascore.com/tournament/football/germany/bundesliga/35',
 'https://www.sofascore.com/tournament/football/poland/ekstraklasa/202',
 'https://www.sofascore.com/tournament/football/israel/israeli-premier-league/266',
 'https://www.sofascore.com/tournament/football/austria/bundesliga/45',
 'https://www.sofascore.com/tournament/football/denmark/superliga/39',
 'https://www.sofascore.com/tournament/football/spain/laliga/8',
 'https://www.sofascore.com/tournament/football/s

In [14]:
# En lugar de usar requests, usa selenium para cada liga
def obtener_soup_liga(url):
    options = Options()
    options.add_argument('--headless')
    options.add_argument('--disable-blink-features=AutomationControlled')
    options.add_experimental_option("excludeSwitches", ["enable-automation"])
    options.add_experimental_option('useAutomationExtension', False)
    
    driver = webdriver.Chrome(options=options)
    driver.execute_script("Object.defineProperty(navigator, 'webdriver', {get: () => undefined})")
    
    try:
        driver.get(url)
        time.sleep(2)  # Esperar a que cargue
        soup = bs(driver.page_source, 'html.parser')
        return soup
    finally:
        driver.quit()

# Probar con la primera liga
soup = obtener_soup_liga(ligas[0])

In [15]:
def get_equipos(soup):  # Cambiar parámetro a soup
    """
    Extrae equipos de un objeto soup de BeautifulSoup
    """
    try:
        equipos = soup.find_all('a')
        equipos = [i for i in equipos if i.get('href', '').startswith('/team/football') and 'world' not in i.get('href', '') and 'europe' not in i.get('href', '')]
        equipos = [i['href'] for i in equipos]
        equipos = list(set(equipos))  # Eliminar duplicados
        equipos = ['https://www.sofascore.com' + i for i in equipos]  # Añadir el dominio
        return equipos
    except Exception as e:
        print(f"Error al obtener equipos: {e}")
        return []


In [16]:
def obtener_todos_equipos_eficiente(ligas):
    """
    Versión más eficiente usando un solo driver
    """
    todos_los_equipos = []
    
    options = Options()
    options.add_argument('--headless')
    options.add_argument('--disable-blink-features=AutomationControlled')
    options.add_experimental_option("excludeSwitches", ["enable-automation"])
    options.add_experimental_option('useAutomationExtension', False)
    
    driver = webdriver.Chrome(options=options)
    driver.execute_script("Object.defineProperty(navigator, 'webdriver', {get: () => undefined})")
    
    try:
        for i, liga_url in enumerate(ligas):
            print(f"Procesando liga {i+1}/{len(ligas)}: {liga_url}")
            
            try:
                driver.get(liga_url)
                time.sleep(2)
                
                soup = bs(driver.page_source, 'html.parser')
                equipos_liga = get_equipos(soup)
                
                print(f"  - Encontrados {len(equipos_liga)} equipos")
                todos_los_equipos.extend(equipos_liga)
                
                time.sleep(1)  # Pausa entre requests
                
            except Exception as e:
                print(f"  - Error en liga {liga_url}: {e}")
                continue
                
    finally:
        driver.quit()
    
    equipos_unicos = list(set(todos_los_equipos))
    print(f"\nTotal de equipos únicos encontrados: {len(equipos_unicos)}")
    
    return equipos_unicos

# Usar la versión eficiente
todos_equipos = obtener_todos_equipos_eficiente(ligas)

Procesando liga 1/24: https://www.sofascore.com/tournament/football/portugal/liga-portugal-betclic/238
  - Encontrados 18 equipos
Procesando liga 2/24: https://www.sofascore.com/tournament/football/hungary/nb-i/187
  - Encontrados 12 equipos
Procesando liga 3/24: https://www.sofascore.com/tournament/football/italy/serie-a/23
  - Encontrados 20 equipos
Procesando liga 4/24: https://www.sofascore.com/tournament/football/cyprus/1-division/171
  - Encontrados 14 equipos
Procesando liga 5/24: https://www.sofascore.com/tournament/football/france/ligue-1/34
  - Encontrados 18 equipos
Procesando liga 6/24: https://www.sofascore.com/tournament/football/turkey/trendyol-super-lig/52
  - Encontrados 19 equipos
Procesando liga 7/24: https://www.sofascore.com/tournament/football/czech-republic/1-liga/172
  - Encontrados 16 equipos
Procesando liga 8/24: https://www.sofascore.com/tournament/football/germany/bundesliga/35
  - Encontrados 18 equipos
Procesando liga 9/24: https://www.sofascore.com/tourna

In [55]:
def obtener_soup_equipo(url):
    """
    Obtiene el soup de la página de un equipo específico.
    """
    options = Options()
    options.add_argument('--headless')
    options.add_argument('--disable-blink-features=AutomationControlled')
    options.add_experimental_option("excludeSwitches", ["enable-automation"])
    options.add_experimental_option('useAutomationExtension', False)
    
    driver = webdriver.Chrome(options=options)
    driver.execute_script("Object.defineProperty(navigator, 'webdriver', {get: () => undefined})")

    
    try:
        driver.get(url+'#tab:squad')
        wait = WebDriverWait(driver, 10)
        soup = bs(driver.page_source, 'html.parser')
        return soup
    finally:
        driver.quit()

In [18]:
def get_jugadores(team_url):
    """
    Obtiene los jugadores de un equipo dado su URL.
    """
    soup = obtener_soup_equipo(team_url)
    
    jugadores = []
    for jugador in soup.find_all('a'):
        if jugador['href'].startswith('/football/player') and 'world' not in jugador['href'] and 'europe' not in jugador['href']:
                jugadores.append('https://www.sofascore.com' + jugador['href'])
    
    return jugadores  # Eliminar duplicados
set(get_jugadores(todos_equipos[0]))

{'https://www.sofascore.com/football/player/alexandros-theocharous/93449',
 'https://www.sofascore.com/football/player/antonis-loizou/2013665',
 'https://www.sofascore.com/football/player/antreas-shikkis/1639232',
 'https://www.sofascore.com/football/player/bourama-fomba/1021271',
 'https://www.sofascore.com/football/player/carlos-peixoto/892221',
 'https://www.sofascore.com/football/player/charalampos-kattirtzis/1427566',
 'https://www.sofascore.com/football/player/christodoulou-georgios/1095926',
 'https://www.sofascore.com/football/player/dimitriou-andreas/1116960',
 'https://www.sofascore.com/football/player/giorgi-loria/37098',
 'https://www.sofascore.com/football/player/giorgos-papacharalampous/1639227',
 'https://www.sofascore.com/football/player/giorgos-pontikos/899027',
 'https://www.sofascore.com/football/player/hristian-foti/933632',
 'https://www.sofascore.com/football/player/joao-sidonio/1161041',
 'https://www.sofascore.com/football/player/jorginho/352680',
 'https://www.

In [19]:
equipos_df = pd.DataFrame(todos_equipos, columns=['url'])
equipos_df.to_csv('equipos_sofascore.csv', index=False)

In [20]:
todos_equipos = pd.read_csv('equipos_sofascore.csv')['url'].tolist()

In [21]:
def obtener_todos_jugadores_secuencial(todos_equipos):
    """
    Obtiene todos los jugadores de todos los equipos de forma secuencial
    """
    todos_los_jugadores = []
    
    for i, equipo_url in enumerate(todos_equipos):
        print(f"Procesando equipo {i+1}/{len(todos_equipos)}: {equipo_url}")
        
        try:
            jugadores_equipo = get_jugadores(equipo_url)
            print(f"  - Encontrados {len(jugadores_equipo)} jugadores")
            todos_los_jugadores.extend(jugadores_equipo)
            
        except Exception as e:
            print(f"  - Error en equipo {equipo_url}: {e}")
            continue
    
    # Eliminar duplicados finales
    jugadores_unicos = list(set(todos_los_jugadores))
    print(f"\nTotal de jugadores únicos encontrados: {len(jugadores_unicos)}")
    
    return jugadores_unicos

# Ejecutar para obtener todos los jugadores
todos_jugadores = obtener_todos_jugadores_secuencial(todos_equipos)  # Limitar a los primeros 10 equipos para pruebas

Procesando equipo 1/353: https://www.sofascore.com/team/football/omonia-aradippou/25542
  - Encontrados 56 jugadores
Procesando equipo 2/353: https://www.sofascore.com/team/football/hatayspor/3091
  - Encontrados 52 jugadores
Procesando equipo 3/353: https://www.sofascore.com/team/football/panserraikos/6347
  - Encontrados 80 jugadores
Procesando equipo 4/353: https://www.sofascore.com/team/football/hamkam/666
  - Encontrados 60 jugadores
Procesando equipo 5/353: https://www.sofascore.com/team/football/antalyaspor/3056
  - Encontrados 60 jugadores
Procesando equipo 6/353: https://www.sofascore.com/team/football/sc-farense/2998
  - Encontrados 52 jugadores
Procesando equipo 7/353: https://www.sofascore.com/team/football/ik-sirius/1793
  - Encontrados 66 jugadores
Procesando equipo 8/353: https://www.sofascore.com/team/football/hnk-hajduk-split/2036
  - Encontrados 58 jugadores
Procesando equipo 9/353: https://www.sofascore.com/team/football/djurgardens-if/1759
  - Encontrados 60 jugador

There was an error managing chromedriver (error sending request for url (https://googlechromelabs.github.io/chrome-for-testing/known-good-versions-with-downloads.json)); using driver found in the cache
Error sending stats to Plausible: error sending request for url (https://plausible.io/api/event)


  - Error en equipo https://www.sofascore.com/team/football/aik/1764: Message: unknown error: net::ERR_NAME_NOT_RESOLVED
  (Session info: chrome=137.0.7151.120)
Stacktrace:
	GetHandleVerifier [0x0x7ff63c8dfe75+79173]
	GetHandleVerifier [0x0x7ff63c8dfed0+79264]
	(No symbol) [0x0x7ff63c699e5a]
	(No symbol) [0x0x7ff63c696874]
	(No symbol) [0x0x7ff63c687589]
	(No symbol) [0x0x7ff63c689331]
	(No symbol) [0x0x7ff63c6878a6]
	(No symbol) [0x0x7ff63c687306]
	(No symbol) [0x0x7ff63c686fca]
	(No symbol) [0x0x7ff63c684b9f]
	(No symbol) [0x0x7ff63c68541c]
	(No symbol) [0x0x7ff63c69de0a]
	(No symbol) [0x0x7ff63c741dfe]
	(No symbol) [0x0x7ff63c71896a]
	(No symbol) [0x0x7ff63c74100d]
	(No symbol) [0x0x7ff63c718743]
	(No symbol) [0x0x7ff63c6e14c1]
	(No symbol) [0x0x7ff63c6e2253]
	GetHandleVerifier [0x0x7ff63cbaa2ad+3004797]
	GetHandleVerifier [0x0x7ff63cba46fd+2981325]
	GetHandleVerifier [0x0x7ff63cbc3350+3107360]
	GetHandleVerifier [0x0x7ff63c8fa9fe+188622]
	GetHandleVerifier [0x0x7ff63c90228f+219487]

There was an error managing chromedriver (error sending request for url (https://googlechromelabs.github.io/chrome-for-testing/known-good-versions-with-downloads.json)); using driver found in the cache


  - Error en equipo https://www.sofascore.com/team/football/ifk-norrkoping/1769: Message: unknown error: net::ERR_NAME_NOT_RESOLVED
  (Session info: chrome=137.0.7151.120)
Stacktrace:
	GetHandleVerifier [0x0x7ff63c8dfe75+79173]
	GetHandleVerifier [0x0x7ff63c8dfed0+79264]
	(No symbol) [0x0x7ff63c699e5a]
	(No symbol) [0x0x7ff63c696874]
	(No symbol) [0x0x7ff63c687589]
	(No symbol) [0x0x7ff63c689331]
	(No symbol) [0x0x7ff63c6878a6]
	(No symbol) [0x0x7ff63c687306]
	(No symbol) [0x0x7ff63c686fca]
	(No symbol) [0x0x7ff63c684b9f]
	(No symbol) [0x0x7ff63c68541c]
	(No symbol) [0x0x7ff63c69de0a]
	(No symbol) [0x0x7ff63c741dfe]
	(No symbol) [0x0x7ff63c71896a]
	(No symbol) [0x0x7ff63c74100d]
	(No symbol) [0x0x7ff63c718743]
	(No symbol) [0x0x7ff63c6e14c1]
	(No symbol) [0x0x7ff63c6e2253]
	GetHandleVerifier [0x0x7ff63cbaa2ad+3004797]
	GetHandleVerifier [0x0x7ff63cba46fd+2981325]
	GetHandleVerifier [0x0x7ff63cbc3350+3107360]
	GetHandleVerifier [0x0x7ff63c8fa9fe+188622]
	GetHandleVerifier [0x0x7ff63c902

...

  - Error en equipo https://www.sofascore.com/team/football/getafe/2859: Message: unknown error: net::ERR_NAME_NOT_RESOLVED
  (Session info: chrome=137.0.7151.120)
Stacktrace:
	GetHandleVerifier [0x0x7ff63c8dfe75+79173]
	GetHandleVerifier [0x0x7ff63c8dfed0+79264]
	(No symbol) [0x0x7ff63c699e5a]
	(No symbol) [0x0x7ff63c696874]
	(No symbol) [0x0x7ff63c687589]
	(No symbol) [0x0x7ff63c689331]
	(No symbol) [0x0x7ff63c6878a6]
	(No symbol) [0x0x7ff63c687306]
	(No symbol) [0x0x7ff63c686fca]
	(No symbol) [0x0x7ff63c684b9f]
	(No symbol) [0x0x7ff63c68541c]
	(No symbol) [0x0x7ff63c69de0a]
	(No symbol) [0x0x7ff63c741dfe]
	(No symbol) [0x0x7ff63c71896a]
	(No symbol) [0x0x7ff63c74100d]
	(No symbol) [0x0x7ff63c718743]
	(No symbol) [0x0x7ff63c6e14c1]
	(No symbol) [0x0x7ff63c6e2253]
	GetHandleVerifier [0x0x7ff63cbaa2ad+3004797]
	GetHandleVerifier [0x0x7ff63cba46fd+2981325]
	GetHandleVerifier [0x0x7ff63cbc3350+3107360]
	GetHandleVerifier [0x0x7ff63c8fa9fe+188622]
	GetHandleVerifier [0x0x7ff63c90228f+2194

There was an error managing chromedriver (error sending request for url (https://googlechromelabs.github.io/chrome-for-testing/known-good-versions-with-downloads.json)); using driver found in the cache


  - Error en equipo https://www.sofascore.com/team/football/olympique-de-marseille/1641: Message: unknown error: net::ERR_INTERNET_DISCONNECTED
  (Session info: chrome=137.0.7151.120)
Stacktrace:
	GetHandleVerifier [0x0x7ff63c8dfe75+79173]
	GetHandleVerifier [0x0x7ff63c8dfed0+79264]
	(No symbol) [0x0x7ff63c699e5a]
	(No symbol) [0x0x7ff63c696874]
	(No symbol) [0x0x7ff63c687589]
	(No symbol) [0x0x7ff63c689331]
	(No symbol) [0x0x7ff63c6878a6]
	(No symbol) [0x0x7ff63c687306]
	(No symbol) [0x0x7ff63c686fca]
	(No symbol) [0x0x7ff63c684b9f]
	(No symbol) [0x0x7ff63c68541c]
	(No symbol) [0x0x7ff63c69de0a]
	(No symbol) [0x0x7ff63c741dfe]
	(No symbol) [0x0x7ff63c71896a]
	(No symbol) [0x0x7ff63c74100d]
	(No symbol) [0x0x7ff63c718743]
	(No symbol) [0x0x7ff63c6e14c1]
	(No symbol) [0x0x7ff63c6e2253]
	GetHandleVerifier [0x0x7ff63cbaa2ad+3004797]
	GetHandleVerifier [0x0x7ff63cba46fd+2981325]
	GetHandleVerifier [0x0x7ff63cbc3350+3107360]
	GetHandleVerifier [0x0x7ff63c8fa9fe+188622]
	GetHandleVerifier [0

There was an error managing chromedriver (error sending request for url (https://googlechromelabs.github.io/chrome-for-testing/known-good-versions-with-downloads.json)); using driver found in the cache


  - Encontrados 64 jugadores
Procesando equipo 133/353: https://www.sofascore.com/team/football/paris-saint-germain/1644
  - Encontrados 56 jugadores
Procesando equipo 134/353: https://www.sofascore.com/team/football/nac-breda/2947
  - Encontrados 58 jugadores
Procesando equipo 135/353: https://www.sofascore.com/team/football/cf-estrela-amadora/3035
  - Encontrados 52 jugadores
Procesando equipo 136/353: https://www.sofascore.com/team/football/rc-lens/1648
  - Encontrados 54 jugadores
Procesando equipo 137/353: https://www.sofascore.com/team/football/aek-athens/3250
  - Encontrados 56 jugadores
Procesando equipo 138/353: https://www.sofascore.com/team/football/piast-gliwice/7918
  - Encontrados 58 jugadores
Procesando equipo 139/353: https://www.sofascore.com/team/football/debreceni-vsc/1921
  - Encontrados 56 jugadores
Procesando equipo 140/353: https://www.sofascore.com/team/football/nk-istra-1961/25529
  - Encontrados 66 jugadores
Procesando equipo 141/353: https://www.sofascore.com

In [3]:
jugadores= pd.DataFrame(todos_jugadores, columns=['url'])
jugadores.to_csv('jugadores_sofascore.csv', index=False)

NameError: name 'todos_jugadores' is not defined

In [4]:
todos_jugadores = pd.read_csv('jugadores_sofascore.csv')['url'].tolist()

In [23]:
def obtener_todos_datos_jugadores(lista_jugadores):
    """
    Obtiene todos los datos de todos los jugadores
    """
    datos_jugadores = []
    errores = []
    
    for i, jugador_url in enumerate(lista_jugadores):
        print(f"Procesando jugador {i+1}/{len(lista_jugadores)}: {jugador_url}")
        
        try:
            options = Options()
            options.add_argument('--headless')
            options.add_argument('--disable-blink-features=AutomationControlled')
            options.add_experimental_option("excludeSwitches", ["enable-automation"])
            options.add_experimental_option('useAutomationExtension', False)
            
            driver = webdriver.Chrome(options=options)
            driver.execute_script("Object.defineProperty(navigator, 'webdriver', {get: () => undefined})")
            
            driver.get(jugador_url)
            wait = WebDriverWait(driver, 2)
            
            # Diccionario para almacenar datos del jugador
            datos_jugador = {
                'url': jugador_url,
                'nombre': 'No encontrado',
                'posicion': 'No encontrada',
                'equipo': 'No encontrado',
                'nacionalidad': 'No encontrada',
                'edad': 'No encontrada',
                'altura': 'No encontrada',
                'valor_mercado': 'No encontrado',
                'ataque': 'No encontrado',
                'tecnica': 'No encontrado',
                'defensa': 'No encontrado',
                'creatividad': 'No encontrada',
                'tactica': 'No encontrada',
                'liga': 'No encontrada',
                'partidos': 'No encontrado',
                'valoracion_media': 'No encontrada'
            }
            
            # 1. Nombre (h1)
            try:
                nombre_element = wait.until(EC.presence_of_element_located((By.XPATH, "//div[contains(@class, 'Box Flex icTHgS fRroAj')]")))
                datos_jugador['nombre'] = nombre_element.text
            except:
                pass
            
            # 2. Posición (usando el XPath que ya funciona)
            try:
                posicion_element = wait.until(EC.presence_of_element_located((By.XPATH, "//div[contains(@class, 'Box jwDcoO')]")))
                datos_jugador['posicion'] = posicion_element.text.replace('POSITION', '').replace('\n', '')
            except:
                pass
            
            # 3. Nacionalidad
            try:
                nacionalidad_element = driver.find_element(By.XPATH, "//div[contains(@class, 'Box Flex ggRYVx doveCn')]")
                datos_jugador['nacionalidad'] = nacionalidad_element.text
            except:
                pass
            
            # 4. Equipo
            try:
                equipo_element = driver.find_element(By.XPATH, "//div[contains(@class, 'Text leMLNz')]")
                datos_jugador['equipo'] = equipo_element.text.strip()
            except:
                pass
            
            # 5. Altura (buscar texto que contenga 'cm')
            try:
                altura_element = driver.find_element(By.XPATH, "//div[contains(text(), 'cm')]")
                datos_jugador['altura'] = altura_element.text.strip()
            except:
                pass
            # 6. Edad (buscar patrones como "23 years old" o solo números cerca de fecha)
            try:
                elementos_edad = driver.find_elements(By.XPATH, "//div[contains(text(), 'yrs') or contains(text(), 'años')]")
                for elemento in elementos_edad:
                    texto = elemento.text.strip()
                    numeros = re.findall(r'\d+', texto)
                    if numeros:
                        edad = numeros[0]
                        if 15 <= int(edad) <= 50:  # Rango razonable de edad
                            datos_jugador['edad'] = edad
                            break
            except:
                pass
            #7. market_value
            try:
                valor_element = driver.find_element(By.XPATH, "//div[contains(@class, 'Text imGAlA')]")
                datos_jugador['valor_mercado'] = valor_element.text.strip()
            except:
                pass
            #8. Ataque
            try:
                ataque_element = driver.find_element(By.XPATH, "//span[contains(@class, 'textStyle_table.small c_surface.s1 lh_20px d_block ta_center')]")
                datos_jugador['ataque'] = ataque_element.text.strip()
            except:
                pass
            #9. Tecnica
            try:
                tecnica_element = driver.find_elements(By.XPATH, "//span[contains(@class, 'textStyle_table.small c_surface.s1 lh_20px d_block ta_center')]")[1]
                datos_jugador['tecnica'] = tecnica_element.text.strip()
            except:
                pass
            #10. Tactica
            try:
                defensa_element = driver.find_elements(By.XPATH, "//span[contains(@class, 'textStyle_table.small c_surface.s1 lh_20px d_block ta_center')]")[2]
                datos_jugador['tactica'] = defensa_element.text.strip()
            except:
                pass
            #11.Defensa
            try:
                defensa_element = driver.find_elements(By.XPATH, "//span[contains(@class, 'textStyle_table.small c_surface.s1 lh_20px d_block ta_center')]")[3]
                datos_jugador['defensa'] = defensa_element.text.strip()
            except:
                pass
            #12. Creatividad
            try:
                creatividad_element = driver.find_elements(By.XPATH, "//span[contains(@class, 'textStyle_table.small c_surface.s1 lh_20px d_block ta_center')]")[4]
                datos_jugador['creatividad'] = creatividad_element.text.strip()
            except:
                pass
            #13. Liga
            try:
                liga_element = driver.find_element(By.XPATH, "//div[contains(@class, 'Text gpUqFG')]")
                datos_jugador['liga'] = liga_element.text.strip()
                if datos_jugador['liga'] == datos_jugador['equipo']:
                    datos_jugador['liga'] = 'No encontrada'
            except:
                pass
            #14. Partidos
            try:
                partidos_element = driver.find_element(By.XPATH, "//span[contains(@class, 'Text fftNCK')]")
                datos_jugador['partidos'] = partidos_element.text.strip()
            except:
                pass
            #15. Valoración media
            try:
                valoracion_element = driver.find_element(By.XPATH, "//span[@aria-valuenow and @role='meter']")
                datos_jugador['valoracion_media'] = valoracion_element.get_attribute('aria-valuenow')
            except:
                pass
            
            datos_jugadores.append(datos_jugador)
            print(datos_jugador.values())

        except Exception as e:
            print(f"  ❌ Error en jugador {jugador_url}: {e}")
            errores.append({'url': jugador_url, 'error': str(e)})
            if 'driver' in locals():
                driver.quit()
            continue
    
    # Guardar resultado final
    df_final = pd.DataFrame(datos_jugadores)
    df_final.to_csv('jugadores_datos_completos.csv', index=False)
    
    # Guardar errores
    if errores:
        df_errores = pd.DataFrame(errores)
        df_errores.to_csv('jugadores_errores.csv', index=False)
    
    print(f"\n🎉 Proceso completado!")
    print(f"✅ Jugadores procesados exitosamente: {len(datos_jugadores)}")
    print(f"❌ Errores encontrados: {len(errores)}")
    
    return df_final

# Ejecutar para todos los jugadores
df_jugadores_completo = obtener_todos_datos_jugadores(todos_jugadores[:2])

Procesando jugador 1/2: https://www.sofascore.com/football/player/tvardovskyi-denys/1127399
dict_values(['https://www.sofascore.com/football/player/tvardovskyi-denys/1127399', 'Denys Tvardovskyi', 'G', 'Shakhtar Donetsk', 'UKR', '22', '206 cm', '310K €', 'No encontrado', 'No encontrado', 'No encontrado', 'No encontrada', 'No encontrada', 'No encontrada', 'No encontrado', 'No encontrada'])
Procesando jugador 2/2: https://www.sofascore.com/football/player/ladislav-almasi/825979
dict_values(['https://www.sofascore.com/football/player/ladislav-almasi/825979', 'Ladislav Almási', 'F', 'Zalaegerszegi TE', 'SVK', '26', '196 cm', '255K €', '56', '45', '36', '42', '47', 'Niké Liga', '13 Appearances', '6.74'])

🎉 Proceso completado!
✅ Jugadores procesados exitosamente: 2
❌ Errores encontrados: 0
